<a href="https://colab.research.google.com/github/JonasVerbickas/test-jupyter/blob/main/NLP_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import os
import nltk
import re
import string
import unicodedata
import time
import pandas as pd

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
porter = nltk.PorterStemmer()

In [20]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [21]:
class StringWithDocId:
  def __init__(self, string, doc_id):
    self.string = string
    self.doc_id = doc_id
  
  def __lt__(token_with_doc_A, token_with_doc_B):
    if token_with_doc_A.string == token_with_doc_B.string:
      return token_with_doc_A.doc_id < token_with_doc_B.doc_id
    else:
      return token_with_doc_A.string < token_with_doc_B.string
    
  def __str__(self):
    return f"{self.string}: {self.doc_id}"

  def __repr__(self):
    return f"{self.string}: {self.doc_id}"

In [22]:
class StringWithDocIdAndPosition(StringWithDocId):
  def __init__(self, string, doc_id, position):
    super().__init__(string, doc_id)
    self.position = position
  
  def __lt__(token_with_doc_A, token_with_doc_B):
    if token_with_doc_A.string != token_with_doc_B.string:
      return token_with_doc_A.string < token_with_doc_B.string
    elif token_with_doc_A.doc_id != token_with_doc_B.doc_id:
      return token_with_doc_A.doc_id < token_with_doc_B.doc_id
    else:
      return token_with_doc_A.position < token_with_doc_B.position

In [23]:
class Posting:
  def __init__(self, doc_id, position, doc_freq):
    self.doc_id = doc_id
    self.position = position
    self.doc_freq = doc_freq

  def __str__(self):
    return f"({self.doc_id} {self.position})"
  
  def __repr__(self):
    return f"({self.doc_id} {self.position})"

In [24]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [91]:
def wikipediaPreprocessing(text):
  text = unicodedata.normalize('NFKC', text)
  text = text.replace('\n', '')
  text = re.sub(r'\[.*?\]+', '', text)
  text = re.sub(r'Contents\s+1\s+Plot.+Plot', '', text) 
  text = re.sub(r'^.+From Wikipedia, the free encyclopedia.+List of episodes', '', text)
  return text

In [106]:
STOP_WORDS = nltk.corpus.stopwords.words('english')
class InvertedIndex:
    """
    Construct Inverted Index
    """
    def __init__(self):
        self.inverted_index = {}
        self.csv_terms = set()

    def __getitem__(self, key):
        return self.inverted_index[key]
      
    def keys(self):
      return self.inverted_index.keys()

    # def appendCSV(self, path):
    #   df = pd.read_csv(path)
    #   set_of_csv_entries = df['name']
    #   # for term in df['name']
    #   processed_csv_entries = self.processListOfTokens(set_of_csv_entries)
    #   # remove any duplicate entries
    #   print(processed_csv_entries[:10])
    #   without_duplicates = set(processed_csv_entries)
    #   self.csv_terms = self.csv_terms.union(without_duplicates)

    def read_data(self, path: str) -> list:
        """
        Read files from a directory and then append the data of each file into a list.
        """
        output = []
        for file in os.listdir(path):
          filename_with_path = os.path.join(path, file)
          if (file[-3:]).lower() == 'txt':
            with open(filename_with_path, 'r') as f:
              output.append(StringWithDocId(f.read(), file))
          # elif (file[-3:]).lower() == 'csv':
          #   self.appendCSV(filename_with_path)
          else:
            print(filename_with_path, "will be skipped")
        print(len(output))
        return output
      
    def processListOfTokens(self, tokenized):
      output = []
      for token in tokenized:
        # 2. ignore stop-words
        if token in STOP_WORDS:
          continue
        # remove punctuation
        # looping in order to remove tokens that are made up out of multiple punctuation characters
        # e.g. '!)' and '!"'
        for char in token:
          if char not in "’“”—¡"+string.punctuation:
            break
        # if the loop didn't break - the entire string is made up out of punctuation characters
        else:
          continue
        # 3. porter stemmer makes everything lowercase as well
        stemmed = porter.stem(token)
        output.append(stemmed)
      return output

    def process_document(self, document: str, wordpunct_tokenize=True) -> list:
        """
        pre-process a document and return a list of its terms
        str->list"""
        # 1. Wikipedia hyperlinks should be removed
        # 4. Use multi-word character/location names from .csv files
        text = wikipediaPreprocessing(document)
        if wordpunct_tokenize:
          tokenized  = nltk.tokenize.wordpunct_tokenize(text)
        else:
          tokenized  = nltk.tokenize.word_tokenize(text)
        # remove stop-words & apply stemming
        filtered_tokens = self.processListOfTokens(tokenized)
        return filtered_tokens
    
    def index_corpus(self, documents: list) -> None:
        """
        index given documents
        list->None"""
        starting_time = time.perf_counter()
        token_list = []
        # 1. Generate token sequence
        for doc in documents:
          curr_doc_id = doc.doc_id
          processed_string = self.process_document(doc.string)
          for i, token in enumerate(processed_string):
            token_with_doc_id_and_pos = StringWithDocIdAndPosition(token, curr_doc_id, i)
            token_list.append(token_with_doc_id_and_pos)
        # 2. Sort
        sorted_token_list = sorted(token_list)
        print("First 10 of sorted_token_list:", sorted_token_list[:10])
        # 3. Convert into dictionary of postings
        for token in sorted_token_list:
          if token.string in self.inverted_index:
            self.inverted_index[token.string].append(Posting(token.doc_id, token.position, 1))
          else:
            self.inverted_index[token.string] = [Posting(token.doc_id, token.position, 1)]
        # Print out some details about the dataset
        total_time_taken = round(time.perf_counter() - starting_time, 4)
        print(f"It took: {total_time_taken} seconds to index the whole corpus.")
        print(f"It has {len(self.inverted_index)} entries in total.")
      
    def dump(self, path: str) -> None:
        """
        provide a dump function to show index entries for a given set of terms        
        """
        if os.path.exists(path) == False:
          print("Path to file you provided doesn't exist")
          return
        with open(path, 'r') as f:
          file_contents = f.read()
          examples = file_contents.split('\n')
          for e in examples:
            processed_e = " ".join(self.process_document(e))
            try:
              print(f'{len(self.inverted_index[processed_e])} matches for {e}:', self.inverted_index[processed_e])
            except KeyError:
              print(e, "was not found")
     
    def proximity_search(self, term1: str, term2: str, window_size: int = 3) -> dict:
        """
        1) check whether given two terms appear within a window
        2) calculate the number of their co-existance in a document
        3) add the document id and the number of matches into a dict
        return the dict"""
        term1 = " ".join(self.process_document(term1))
        term2 = " ".join(self.process_document(term2))
        documents_containing_both_terms = []
        term1_i = 0
        term2_i = 0
        while term1_i < len(self.inverted_index[term1]) and term2_i < len(self.inverted_index[term2]):
          if self.inverted_index[term1][term1_i].doc_id == self.inverted_index[term2][term2_i].doc_id:
            if abs(self.inverted_index[term1][term1_i].position - self.inverted_index[term2][term2_i].position) <= window_size:
              documents_containing_both_terms.append((str(self.inverted_index[term1][term1_i]), str(self.inverted_index[term2][term2_i])))
            term1_i += 1
            term2_i += 1
          else:
            if self.inverted_index[term1][term1_i].doc_id < self.inverted_index[term2][term2_i].doc_id:
              term1_i += 1
            else:
              term2_i += 1
        return documents_containing_both_terms

In [107]:
def testReadData():
  index = InvertedIndex()
  corpus = index.read_data('/content/drive/MyDrive/Colab Notebooks/Simpsons2022')
  return (corpus[0]).string
testReadData()

/content/drive/MyDrive/Colab Notebooks/Simpsons2022/simpsons_characters - row.csv will be skipped
/content/drive/MyDrive/Colab Notebooks/Simpsons2022/simpsons_locations row.csv will be skipped
118


'Mr. Lisa Goes to Washington\nFrom Wikipedia, the free encyclopedia\n\n\n\nJump to navigation\nJump to search\n"Mr. Lisa Goes to Washington"\nThe Simpsons\xa0episode\nEpisode\xa0no.\nSeason\xa03\nEpisode 2\nDirected by\nWes Archer\nWritten by\nGeorge Meyer\nProduction code\n8F01\nOriginal air date\nSeptember 26, 1991[1]\n\nEpisode features\nChalkboard gag\n"Spitwads are not free speech"\nCouch gag\nThe family sits down and Homer pulls\xa0Santa\'s Little Helper\xa0from under him.\nCommentary\nMatt Groening\nAl Jean\nMike Reiss\nJulie Kavner\nWes Archer\nDavid Silverman\nEpisode chronology\n←\xa0Previous\n"Stark Raving Dad"\nNext\xa0→\n"When Flanders Failed"\nThe Simpsons\xa0(season 3)\nList of episodes\n"Mr. Lisa Goes to Washington" is the second episode of the\xa0third season\xa0of the American animated television series\xa0The Simpsons. It originally aired on the\xa0Fox network\xa0in the United States on September 26, 1991. In the episode,\xa0Lisa\xa0wins a patriotic essay contest abo

In [108]:
def testTokenzination():
    index = InvertedIndex() # initilaise the index
    corpus = index.read_data('/content/drive/MyDrive/Colab Notebooks/Simpsons2022') # specify the directory path in which files are located
    text = (corpus[0]).string
    start_time = time.perf_counter()
    tokenized = index.process_document(text)
    print("Time taken", time.perf_counter()-start_time)
    return tokenized
testTokenzination()[:60]

/content/drive/MyDrive/Colab Notebooks/Simpsons2022/simpsons_characters - row.csv will be skipped
/content/drive/MyDrive/Colab Notebooks/Simpsons2022/simpsons_locations row.csv will be skipped
118
Time taken 0.056041265000203566


['mr',
 'lisa',
 'goe',
 'washington',
 'second',
 'episod',
 'third',
 'season',
 'american',
 'anim',
 'televis',
 'seri',
 'the',
 'simpson',
 'it',
 'origin',
 'air',
 'fox',
 'network',
 'unit',
 'state',
 'septemb',
 '26',
 '1991',
 'in',
 'episod',
 'lisa',
 'win',
 'patriot',
 'essay',
 'contest',
 'unit',
 'state',
 'she',
 'famili',
 'attend',
 'contest',
 'final',
 'washington',
 'd',
 'c',
 'dismay',
 'watch',
 'congressman',
 'accept',
 'bribe',
 'lisa',
 'lose',
 'contest',
 'pen',
 'scath',
 'screed',
 'condemn',
 'govern',
 'system',
 'corrupt',
 'congressman',
 'jail',
 'remov',
 'offic']

In [109]:
def testTokenzination():
    index = InvertedIndex() # initilaise the index
    corpus = index.read_data('/content/drive/MyDrive/Colab Notebooks/Simpsons2022') # specify the directory path in which files are located
    text = (corpus[0]).string
    start_time = time.perf_counter()
    tokenized = index.process_document(text, False)
    print("Time taken", time.perf_counter()-start_time)
    return tokenized
testTokenzination()[:60]

/content/drive/MyDrive/Colab Notebooks/Simpsons2022/simpsons_characters - row.csv will be skipped
/content/drive/MyDrive/Colab Notebooks/Simpsons2022/simpsons_locations row.csv will be skipped
118
Time taken 0.08179863599980308


['mr.',
 'lisa',
 'goe',
 'washington',
 'second',
 'episod',
 'third',
 'season',
 'american',
 'anim',
 'televis',
 'seri',
 'the',
 'simpson',
 'it',
 'origin',
 'air',
 'fox',
 'network',
 'unit',
 'state',
 'septemb',
 '26',
 '1991',
 'in',
 'episod',
 'lisa',
 'win',
 'patriot',
 'essay',
 'contest',
 'unit',
 'state',
 'she',
 'famili',
 'attend',
 'contest',
 'final',
 'washington',
 'd.c.',
 'dismay',
 'watch',
 'congressman',
 'accept',
 'bribe',
 'lisa',
 'lose',
 'contest',
 'pen',
 'scath',
 'screed',
 'condemn',
 'govern',
 'system',
 'corrupt',
 'congressman',
 'jail',
 'remov',
 'offic',
 'restor']

In [110]:
def main():
    "main call function"
    index = InvertedIndex() # initilaise the index
    corpus = index.read_data('/content/drive/MyDrive/Colab Notebooks/Simpsons2022') # specify the directory path in which files are located
    index.index_corpus(corpus) # index documents/corpus
    return index
    
index = main()

/content/drive/MyDrive/Colab Notebooks/Simpsons2022/simpsons_characters - row.csv will be skipped
/content/drive/MyDrive/Colab Notebooks/Simpsons2022/simpsons_locations row.csv will be skipped
118
First 10 of sorted_token_list: [0: 4.2.txt, 0: 4.3.txt, 0: 4.4.txt, 0: 5.21.txt, 0: 5.21.txt, 0: 5.8.txt, 0: 5.8.txt, 0: 6.4.txt, 0: 7.3.txt, 0: 7.3.txt]
It took: 4.8645 seconds to index the whole corpus.
It has 9986 entries in total.


In [111]:
index.dump("/content/drive/MyDrive/Colab Notebooks/26957722.txt")

1172 matches for Bart: [(3.1.txt 42), (3.1.txt 212), (3.1.txt 245), (3.1.txt 299), (3.1.txt 322), (3.1.txt 327), (3.1.txt 341), (3.1.txt 356), (3.1.txt 512), (3.1.txt 523), (3.1.txt 796), (3.1.txt 799), (3.1.txt 1041), (3.1.txt 1065), (3.1.txt 1178), (3.1.txt 1762), (3.1.txt 1798), (3.10.txt 907), (3.10.txt 1085), (3.10.txt 1220), (3.10.txt 1306), (3.11.txt 352), (3.11.txt 553), (3.11.txt 655), (3.11.txt 686), (3.12.txt 39), (3.12.txt 45), (3.12.txt 124), (3.12.txt 153), (3.12.txt 163), (3.12.txt 202), (3.12.txt 207), (3.12.txt 421), (3.12.txt 424), (3.12.txt 443), (3.12.txt 505), (3.12.txt 614), (3.13.txt 1), (3.13.txt 24), (3.13.txt 51), (3.13.txt 62), (3.13.txt 99), (3.13.txt 141), (3.13.txt 145), (3.13.txt 156), (3.13.txt 181), (3.13.txt 232), (3.13.txt 244), (3.13.txt 251), (3.13.txt 263), (3.13.txt 270), (3.13.txt 294), (3.13.txt 307), (3.13.txt 324), (3.13.txt 578), (3.13.txt 669), (3.13.txt 692), (3.13.txt 758), (3.13.txt 801), (3.13.txt 884), (3.13.txt 889), (3.13.txt 924), (3

In [112]:
index.proximity_search('award', 'winning', 100)

[('(3.13.txt 789)', '(3.13.txt 842)'),
 ('(3.21.txt 380)', '(3.21.txt 378)'),
 ('(3.21.txt 381)', '(3.21.txt 395)'),
 ('(3.6.txt 744)', '(3.6.txt 794)'),
 ('(3.6.txt 778)', '(3.6.txt 805)'),
 ('(4.19.txt 159)', '(4.19.txt 220)'),
 ('(4.19.txt 174)', '(4.19.txt 245)'),
 ('(5.1.txt 314)', '(5.1.txt 312)'),
 ('(5.15.txt 131)', '(5.15.txt 128)'),
 ('(5.17.txt 29)', '(5.17.txt 26)'),
 ('(5.9.txt 363)', '(5.9.txt 364)'),
 ('(6.19.txt 55)', '(6.19.txt 65)'),
 ('(6.2.txt 180)', '(6.2.txt 154)'),
 ('(6.22.txt 1030)', '(6.22.txt 1039)'),
 ('(6.7.txt 440)', '(6.7.txt 441)'),
 ('(7.12.txt 595)', '(7.12.txt 563)'),
 ('(7.8.txt 851)', '(7.8.txt 874)')]